# Предполагаемые шаги

1. просуммировать помесячно количество денег и попробавть предсказать на основе этого
2. Оставить деньги на ежедневной основе
3. Попробовать раскурить идею с МСС кодом, поскольку это классификация основных транзакций => что может передать некоторый патерн персонажа

# Импорт необходимых библиотек 

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import phik

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import (StratifiedKFold, cross_validate, cross_val_score, train_test_split, GridSearchCV)
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier

# Чтение файлов

In [6]:
df = pd.read_parquet("df_transaction.pa")
display(df.head())

train = pd.read_parquet("train.pa")
display(train.head())

,client_num,date_time,mcc_code,merchant_name,amount
0,0,2024-07-18 16:04:00,8099,a011100358d0f73ea8f3e860ef5564e3ba9cb217b7b90c...,2900
1,0,2024-07-22 16:31:00,5411,f3855606fc7244ec2f37ea01a4b2b66933d0e965bf4aec...,455
2,0,2024-07-24 16:23:00,5541,786270fa33ad4ac2a3c0e52e888005aa7f98beadbf8986...,1003
3,0,2024-07-28 15:51:00,5691,54887ad4a8df7e260a3ac85e59128a947c50d4423f6330...,1480
4,0,2024-07-28 18:00:00,5331,21617559a372c7cca155208c87be6c84ce97b5f8775589...,88


,client_num,target
0,94779,3
1,17279,0
2,5717,2
3,27471,1
4,72725,0


In [7]:
data_iter_1 = df[['client_num', 'date_time', 'amount']].copy()
train_1 = train.copy()

In [8]:
data_iter_1['date_time'] = pd.to_datetime(data_iter_1['date_time']).dt.month

In [9]:
data_iter_1.head()

,client_num,date_time,amount
0,0,7,2900
1,0,7,455
2,0,7,1003
3,0,7,1480
4,0,7,88


In [10]:
pivot_data_iter_1 = data_iter_1.pivot_table(columns='date_time', index='client_num', aggfunc='sum').reset_index()
pivot_data_iter_1.columns = pivot_data_iter_1.columns.droplevel()

In [11]:
pivot_data_iter_1 = pivot_data_iter_1.rename(columns={'' : 'client_num'})

In [12]:
pivot_data_iter_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109143 entries, 0 to 109142
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   client_num  109143 non-null  int64  
 1   7           95267 non-null   float64
 2   8           101294 non-null  float64
 3   9           102025 non-null  float64
 4   10          4 non-null       float64
dtypes: float64(4), int64(1)
memory usage: 4.2 MB


In [13]:
#pivot_data_iter_1 = pivot_data_iter_1.drop(columns = 10, axis =1)

In [14]:
pivot_data_iter_1.head()

date_time,client_num,7,8,9,10
0,0,7261.0,56962.0,42712.0,NaN
1,1,422749.0,236673.0,204456.0,NaN
2,2,114647.0,80550.0,148911.0,NaN
3,3,1483913.0,68103.0,69809.0,NaN
4,4,91422.0,63710.0,44664.0,NaN


# Вторая итерация 
Проверяем следующую информацию по шагам. 
1. смотрим по датам
2. к датам добавляем месячную трату
3. каким то образом пытаемся внедрить CVM?????????????????????????

In [16]:
second_train = df[df['client_num'].isin(train['client_num'])].reset_index(drop = True)

In [17]:
second_test = df[~df['client_num'].isin(train['client_num'])].reset_index(drop = True)

In [18]:
# Проверка разделения
(len(second_train['client_num'].unique()) + len(second_test['client_num'].unique())) == (len(df['client_num'].unique()))

True

In [19]:
second_train = second_train.drop(columns=['mcc_code', 'merchant_name'], axis=1)
second_test = second_test.drop(columns=['mcc_code', 'merchant_name'], axis=1)

In [20]:
def preparation_matrix_second (df):
    df['date_time'] = df['date_time'].dt.date #Преобразовали в дату для группировки 
    grouped = df.groupby(['client_num', 'date_time'])['amount'].agg('sum').reset_index()

    client_nums = grouped['client_num'].unique()
    date_range = pd.date_range(start='2024-07-01', end='2024-09-30', freq='D')  
    index = pd.MultiIndex.from_product([client_nums, date_range], names=['client_num', 'date_time'])
    result_date = pd.DataFrame(index=index).reset_index()
    result_date['date_time'] = pd.to_datetime(result_date['date_time']).dt.date
    grouped['date_time'] = pd.to_datetime(grouped['date_time']).dt.date
    df = result_date.merge(grouped, on =['client_num', 'date_time'], how = 'left')

    df = df.pivot_table(columns = 'date_time', index = 'client_num', values='amount', aggfunc='sum').reset_index()
    
    return(df)

In [21]:
second_train_prepered = preparation_matrix_second(second_train)

In [22]:
second_test_prepered = preparation_matrix_second(second_test)

In [23]:
second_train_prepered = second_train_prepered.merge(pivot_data_iter_1, on='client_num', how='left').fillna(0)
second_test_prepered = second_test_prepered.merge(pivot_data_iter_1, on='client_num', how='left').fillna(0)

Сделаем таблицу с поиском по MCC кодам

In [25]:
mcc_code = df.copy()

In [26]:
mcc_code['date_time'] = mcc_code['date_time'].dt.month

In [27]:
mcc_table = mcc_code.pivot_table(columns=['date_time', 'mcc_code'], index='client_num', values='merchant_name', aggfunc='count').reset_index().fillna(0)

In [28]:
jule = mcc_table[7]
august = mcc_table[8]
september = mcc_table[9]

In [29]:
jule.columns = ['jule_' + col for col in jule.columns]
august.columns = ['august_' + col for col in august.columns]
september.columns = ['september_' + col for col in september.columns]

jule = jule.reset_index()
august = august.reset_index()
september = september.reset_index()

In [30]:
mcc_table = jule.merge(august, on='index', how='left')
mcc_table = mcc_table.merge(september, on='index', how='left')

mcc_table = mcc_table.rename(columns = {'index':'client_num'})

In [31]:
second_train_prepered_mcc = second_train_prepered.merge(mcc_table, on='client_num', how='left')
second_test_prepered_mcc = second_test_prepered.merge(mcc_table, on='client_num', how='left')

Подготовили таблицы, объеденяем и после этого смотрим на метрику на кросс валидации

In [33]:
second_XY_train_prepered = second_train_prepered_mcc.merge(train, on ='client_num', how='left')

In [34]:
X_train_2 = second_XY_train_prepered.drop(columns=['client_num', 'target'], axis=1)
y_train_2 = second_XY_train_prepered['target']

In [35]:
X_test_2 = second_test_prepered_mcc.drop(columns='client_num', axis=1)

In [36]:
X_train_2.columns = X_train_2.columns.astype(str)
X_test_2.columns = X_test_2.columns.astype(str)

In [37]:
X_train_2

,2024-07-01,2024-07-02,2024-07-03,2024-07-04,2024-07-05,2024-07-06,2024-07-07,2024-07-08,2024-07-09,2024-07-10,...,september_8734,september_8911,september_8931,september_8999,september_9222,september_9311,september_9390,september_9399,september_9402,september_9406
0,18987.0,132614.0,1692.0,3892.0,19136.0,0.0,0.0,684.0,75188.0,2522.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2092.0,1537.0,5452.0,1627.0,1232.0,4997.0,2412.0,8065.0,1322.0,5117.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2709.0,250.0,0.0,6273.0,1300653.0,2000.0,3765.0,252.0,272.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0
3,0.0,760.0,3571.0,345.0,55378.0,0.0,0.0,638.0,605.0,959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2269.0,619.0,90.0,30.0,1717.0,880.0,0.0,30.0,60.0,427.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,5881.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# scaler = StandardScaler()

# # Масштабирование данных
# X_train_scaled = scaler.fit_transform(X_train_2)
# X_test_scaled = scaler.transform(X_test_2)

# # Преобразование в DataFrame с сохранением колонок и индексов
# X_train_2 = pd.DataFrame(X_train_scaled, columns=X_train_2.columns, index=X_train_2.index)
# X_test_2 = pd.DataFrame(X_test_scaled, columns=X_test_2.columns, index=X_test_2.index)

In [39]:
X_train_2

,2024-07-01,2024-07-02,2024-07-03,2024-07-04,2024-07-05,2024-07-06,2024-07-07,2024-07-08,2024-07-09,2024-07-10,...,september_8734,september_8911,september_8931,september_8999,september_9222,september_9311,september_9390,september_9399,september_9402,september_9406
0,18987.0,132614.0,1692.0,3892.0,19136.0,0.0,0.0,684.0,75188.0,2522.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2092.0,1537.0,5452.0,1627.0,1232.0,4997.0,2412.0,8065.0,1322.0,5117.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2709.0,250.0,0.0,6273.0,1300653.0,2000.0,3765.0,252.0,272.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0
3,0.0,760.0,3571.0,345.0,55378.0,0.0,0.0,638.0,605.0,959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2269.0,619.0,90.0,30.0,1717.0,880.0,0.0,30.0,60.0,427.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,5881.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
model_2 = CatBoostClassifier(iterations = 2000, border_count = None)

# Оценка модели с использованием кросс-валидации и метрики F1
f1_scores = cross_val_score(model_2, X_train_2, y_train_2, cv=5, scoring='f1_weighted')

# Выводим метрики F1 для каждой итерации кросс-валидации
print(f"F1 Scores for each fold: {f1_scores}")
print(f"Mean F1 Score: {np.mean(f1_scores)}")

Learning rate set to 0.054089
0:	learn: 1.9232364	total: 536ms	remaining: 17m 51s
1:	learn: 1.9038029	total: 869ms	remaining: 14m 27s
2:	learn: 1.8864735	total: 1.3s	remaining: 14m 25s
3:	learn: 1.8711214	total: 1.72s	remaining: 14m 19s
4:	learn: 1.8576609	total: 2.07s	remaining: 13m 45s
5:	learn: 1.8457576	total: 2.38s	remaining: 13m 9s
6:	learn: 1.8335315	total: 2.73s	remaining: 12m 56s
7:	learn: 1.8232839	total: 3.06s	remaining: 12m 41s
8:	learn: 1.8142605	total: 3.35s	remaining: 12m 21s
9:	learn: 1.8051813	total: 3.68s	remaining: 12m 11s
10:	learn: 1.7976446	total: 4s	remaining: 12m 2s
11:	learn: 1.7898867	total: 4.33s	remaining: 11m 58s
12:	learn: 1.7835618	total: 4.63s	remaining: 11m 48s
13:	learn: 1.7772316	total: 4.98s	remaining: 11m 46s
14:	learn: 1.7713953	total: 5.28s	remaining: 11m 39s
15:	learn: 1.7657696	total: 5.59s	remaining: 11m 32s
16:	learn: 1.7609454	total: 5.89s	remaining: 11m 26s
17:	learn: 1.7560843	total: 6.21s	remaining: 11m 23s
18:	learn: 1.7518573	total: 6.53

In [42]:
model_2.fit(X_train_2, y_train_2)

Learning rate set to 0.054708
0:	learn: 1.9234707	total: 443ms	remaining: 14m 46s
1:	learn: 1.9036497	total: 874ms	remaining: 14m 33s
2:	learn: 1.8859443	total: 1.29s	remaining: 14m 20s
3:	learn: 1.8709622	total: 1.66s	remaining: 13m 48s
4:	learn: 1.8572332	total: 2.06s	remaining: 13m 41s
5:	learn: 1.8440570	total: 2.46s	remaining: 13m 37s
6:	learn: 1.8322403	total: 2.86s	remaining: 13m 34s
7:	learn: 1.8218051	total: 3.3s	remaining: 13m 41s
8:	learn: 1.8123146	total: 3.7s	remaining: 13m 38s
9:	learn: 1.8037603	total: 4.09s	remaining: 13m 33s
10:	learn: 1.7958121	total: 4.53s	remaining: 13m 39s
11:	learn: 1.7891249	total: 4.93s	remaining: 13m 37s
12:	learn: 1.7824607	total: 5.36s	remaining: 13m 39s
13:	learn: 1.7762354	total: 5.76s	remaining: 13m 36s
14:	learn: 1.7709359	total: 6.14s	remaining: 13m 33s
15:	learn: 1.7660718	total: 6.55s	remaining: 13m 32s
16:	learn: 1.7608861	total: 6.99s	remaining: 13m 35s
17:	learn: 1.7563840	total: 7.39s	remaining: 13m 33s
18:	learn: 1.7525827	total: 

In [43]:
f1_weighted = f1_score(y_train_2, model_2.predict(X_train_2), average='weighted')
print(f1_weighted)
# Прогнозирование на тестовом наборе данных
y_pred_2 = model_2.predict(X_test_2)

0.4785600174665825


In [44]:
y_pred_2 = pd.Series(y_pred_2.flatten(), name='predictions')  

y_pred_2 = pd.concat([second_test_prepered, y_pred_2], axis=1)

In [45]:
y_pred_2 = y_pred_2[['client_num', 'predictions']]

In [46]:
y_pred_2 = y_pred_2.rename(columns = {'predictions' : 'target'})

In [47]:
y_pred_2.to_csv("full_table_everyday_plus_month_cmm.csv", index=False, encoding='utf-8')

# Выводы

1. По первичному анализу можно сделать выводы что люди с категорией 6 тратят больше чем люди с категорией 0
2. проведен анализ и предсказание по 3м месяцам (сумма за каждый месяц) в итоге на линейной регрессии получили значения метрики > 0.18 => что метрика на тестовой выборке 1.06.
3. Применена более сложная модель для вычисления - catboost, результат применения которого дал значения метрики на кросс валидации = 0.24, в системе метрика улучшилась до 0.91
4. катбуст без обрезания данных дал лучшее качество на 0.5% 

In [ ]:
'model': [CatBoostClassifier(random_state=RANDOM_STATE)], 
        'model__iterations': [100, 500, 1000, 2000, 3000],
        # #'model__depth': [6, 8, 10],
        # 'model__learning_rate': [0.01, 0.05, 0.1],
        # #'model__l2_leaf_reg': [1, 3, 5, 7],
        'model__loss_function': ['MultiClass']

In [ ]:
Обычный катбуст - 0.30840766575821993   0.79014
iterations = 2000 - 0.3086364537202143   0.79014